In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.linalg
import os

In [3]:
def generate_mesh(x_min, x_max, y_min, y_max, n_cells_x, n_cells_y):
    
    n_vertices_x = n_cells_x + 1
    n_vertices_y = n_cells_y + 1
    n_cells = n_cells_x * n_cells_y

    x, y = np.meshgrid(np.linspace(x_min, x_max, n_vertices_x), np.linspace(y_min, y_max, n_vertices_y))
    vertices = np.hstack([x.reshape(-1, 1), y.reshape(-1, 1)])

    cells = np.zeros([n_cells, 4], dtype=np.int64)
    for j in range(0, n_cells_y):
        for i in range(0, n_cells_x):
            k = i + n_cells_x*j  

            cells[k, 0] = (i) + (n_cells_x + 1)*(j)  # the linear index of the lower left corner of the element
            cells[k, 1] = (i+1) + (n_cells_x + 1)*(j)  # the linear index of the lower right corner of the element
            cells[k, 2] = (i) + (n_cells_x + 1)*(j+1)  # the linear index of the upper right corner of the element
            cells[k, 3] = (i+1) + (n_cells_x + 1)*(j+1)  # the linear index of the upper right corner of the element

    return vertices, cells

In [4]:
def compute_local_mass_matrix():
    """Int_omega BiBk dx = Int_Omega BjBl dy; Multiply by h"""
    M_local = np.zeros([2, 2])
    M_local[0, 0] = 0.5
    M_local[1, 1] = 0.5
    
    return M_local

def compute_local_stiffness_matrix():
    """Int_omega Bix Bkx dx = Int_Omega Bjy Bly dy; Multiply by 1/h"""
    N_local = np.ones([2, 2])
    N_local[0, 1] = -1.0
    N_local[1, 0] = -1.0
    
    return N_local

def compute_local_advection_matrix():
    """Int_omega Bix Bk dx = Int_Omega Bjy Bl dy; Multiply by 1"""
    A_local = 0.5*np.ones([2, 2])
    A_local[0, 0] = -0.5
    A_local[1, 0] = -0.5
    
    return A_local

def compute_A1_and_A2_local():
    A1_A2 = np.zeros([4, 4])
    S_local_1D = compute_local_stiffness_matrix()
    M_local_1D = compute_local_mass_matrix()
    A1_A2 = np.kron(S_local_1D, M_local_1D) + np.kron(M_local_1D, S_local_1D)
    return A1_A2

def compute_global_A_1_A_2(vertices, cells):

    n_cells = cells.shape[0]
    n_vertices = vertices.shape[0]

    N_row_idx = np.zeros([n_cells, 4, 4])
    N_col_idx = np.zeros([n_cells, 4, 4]) 
    N_data = np.zeros([n_cells, 4, 4])

    delta_x = (vertices[cells[:, 1], 0] - vertices[cells[:, 0], 0]).flatten()
    delta_y = (vertices[cells[:, 2], 1] - vertices[cells[:, 0], 1]).flatten()

    A_local = compute_A1_and_A2_local()

    for cell_idx, cell in enumerate(cells):
        col_idx, row_idx = np.meshgrid(cell, cell)
        N_row_idx[cell_idx, :, :] = row_idx
        N_col_idx[cell_idx, :, :] = col_idx
        N_data[cell_idx, :, :] = A_local
    
    A_global = scipy.sparse.csr_array((N_data.flatten(), (N_row_idx.flatten(), N_col_idx.flatten())), shape=(n_vertices, n_vertices))
    
    return A_global

def compute_Forcing(n_vertices):
    return np.ones(2*n_vertices.shape[0])

In [5]:
def find_overlap_indices(vertices_full, cells_full, x_min_sub, x_max_sub):
    # Collect all cells within the x-range of the subdomain
    overlapping_indices = []

    for index, cell in enumerate(cells_full):
        # Get all x coordinates of the vertices in the current cell
        x_coords = vertices_full[cell, 0]
        
        # Check if any of the x coordinates of the cell vertices are within the subdomain range
        if any((x >= x_min_sub) and (x <= x_max_sub) for x in x_coords):
            overlapping_indices.append(index)
    return overlapping_indices

def find_overlap_indices(vertices_full, cells_full, x_min_sub, x_max_sub):
    # Collect all cells within the x-range of the subdomain
    overlapping_indices = []
    
    for index, cell in enumerate(cells_full):
        # Get all x coordinates of the vertices in the current cell
        x_coords = vertices_full[cell, 0]
        x_coords = x_coords.reshape(2,2)
        # Check if any of the x coordinates of the cell vertices are within the subdomain range
        if x_coords[0,0] == x_min_sub or x_coords[1,1] == x_max_sub:
            overlapping_indices.append(index)
    print(overlapping_indices)
    return overlapping_indices

# To find local indices for subdomains, you must map global indices to local
def map_to_local(full_indices, local_cells):
    local_indices = []
    # Create a mapping from global index to local index
    global_to_local = {tuple(sorted(cells_full_domain[i])): i for i in range(len(cells_full_domain))}
    for index in full_indices:
        cell = tuple(sorted(cells_full_domain[index]))
        if cell in global_to_local:
            local_indices.append(global_to_local[cell])
    return local_indices

def set_boundary_conditions(n_cells_x, n_cells_y, F, matrices, phi_left, phi_right, phi_bottom, phi_top):

    # Left boundary
    i_idx = np.zeros(n_cells_y + 1, dtype=np.int64)
    j_idx = np.arange(0, n_cells_y + 1)
    matrices = handle_boundary(i_idx, j_idx, n_cells_x, F, matrices, phi_left)

    # Right boundary
    i_idx = n_cells_x*np.ones(n_cells_y + 1, dtype=np.int64)
    j_idx = np.arange(0, n_cells_y + 1)
    matrices = handle_boundary(i_idx, j_idx, n_cells_x, F, matrices, phi_right)

    # Bottom boundary
    i_idx = np.arange(0, n_cells_x + 1)
    j_idx = np.zeros(n_cells_x + 1, dtype=np.int64)
    matrices = handle_boundary(i_idx, j_idx, n_cells_x, F, matrices, phi_bottom)

    # Top boundary
    i_idx = np.arange(0, n_cells_x + 1)
    j_idx = n_cells_y*np.ones(n_cells_x + 1, dtype=np.int64)
    matrices = handle_boundary(i_idx, j_idx, n_cells_x, F, matrices, phi_top)


def handle_boundary(i_idx, j_idx, n_cells_x, F, matrices, phi):
        
        basis_indices = i_idx + (n_cells_x + 1)*j_idx
        for matrix in matrices:
            matrix[basis_indices, :] = 0.0
            matrix[basis_indices, basis_indices] = 1.0
        
        F[basis_indices] = F[basis_indices+int(len(F)/2)] = phi
        return matrices

def handle_Schwartz_boundary(i_idx, j_idx, n_cells_x, F, matrices, phi):
        basis_indices = i_idx + (n_cells_x + 1)*j_idx
        
        for matrix in matrices:
            print(matrix.shape)
            matrix[basis_indices, :] = 0.0
            matrix[basis_indices, basis_indices] = 1.0
        F[basis_indices] = phi
        return matrices
    
def plot_results(X, Y, U, V, vertices, u1, u2, n_cells_x, n_cells_y):
    if not os.path.exists('figures'):
        os.makedirs('figures')


    """plt.figure(figsize=(10, 6))
    plt.streamplot(X, Y, U, V, density=1)
    plt.title(f"Streamplot of Velocity Field (u_x, u_1) - Cells: {n_cells_x}x{n_cells_y}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()
    #plt.savefig(f'figures/streamplot_{n_cells_x}x{n_cells_y}.pdf')
    #plt.close()"""

    plt.figure(figsize=(10, 6))
    plt.pcolormesh(vertices[:, 0].reshape(n_cells_y+1, n_cells_x+1), vertices[:, 1].reshape(n_cells_y+1, n_cells_x+1), u1.reshape(n_cells_y+1, n_cells_x+1))
    plt.colorbar(label="U1")
    plt.title(f"U1 Field - Cells: {n_cells_x}x{n_cells_y}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

    #plt.savefig(f'figures/u1_field_{n_cells_x}x{n_cells_y}.pdf')
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.pcolormesh(vertices[:, 0].reshape(n_cells_y+1, n_cells_x+1), vertices[:, 1].reshape(n_cells_y+1, n_cells_x+1), u2.reshape(n_cells_y+1, n_cells_x+1))
    plt.colorbar(label="U2")
    plt.title(f"U2 Field - Cells: {n_cells_x}x{n_cells_y}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

    # plt.savefig(f'figures/u2_field_{n_cells_x}x{n_cells_y}.pdf')
    plt.close()

In [6]:
x_min = y_min = 0.0
x_max = 2.0
y_max = 1.0
n_cells_x = 20
n_cells_y = 10
overlap = 0.1
phi_left =  phi_right = phi_bottom = phi_top = 0.0

vertices_full_domain, cells_full_domain = generate_mesh(x_min, x_max, y_min, y_max, n_cells_x, n_cells_y)

delta_x = (vertices_full_domain[cells_full_domain[:, 1], 0] - vertices_full_domain[cells_full_domain[:, 0], 0]).flatten()
delta_y = (vertices_full_domain[cells_full_domain[:, 2], 1] - vertices_full_domain[cells_full_domain[:, 0], 1]).flatten()

#Subdomain 1
i = int((n_cells_x/2 + overlap * n_cells_x))
x_min_S1 = 0.0
x_max_S1 = sum(delta_x[:i])
n_cells_x_S1 = i
vertices_S1, cells_S1 = generate_mesh(x_min_S1, x_max_S1, y_min, y_max, n_cells_x_S1, n_cells_y)

#Subdomain 2
j = int((n_cells_x/2 - overlap * n_cells_x))
x_min_S2 = sum(delta_x[:j])
x_max_S2 = 2.0
n_cells_x_S2 = n_cells_x-j
vertices_S2, cells_S2 = generate_mesh(x_min_S2, x_max_S2, y_min, y_max, n_cells_x_S2, n_cells_y)

#Forcing term
F1 = compute_Forcing(vertices_S1)
F2 = compute_Forcing(vertices_S2)

A1_S1 = compute_global_A_1_A_2(vertices_S1, cells_S1)
A2_S1 = A1_S1.copy()

matrices = [A1_S1, A2_S1]

set_boundary_conditions(n_cells_x_S1, n_cells_y, F1, matrices, phi_left, phi_right, phi_bottom, phi_top)

S = scipy.sparse.bmat([[A1_S1, None], [None, A2_S1]]).tocsr()

u = scipy.sparse.linalg.spsolve(S, F1)

print(len(F1))
























"""



u1 = u[:vertices_S1.shape[0]]; u2 = u[vertices_S1.shape[0]:2*vertices_S1.shape[0]]

# Find indices of S1 cells in the full domain that overlap with S2
overlap_S1_in_S2 = find_overlap_indices(vertices_full_domain, cells_full_domain, x_min_S2, x_max_S2)
# Find indices of S2 cells in the full domain that overlap with S1
overlap_S2_in_S1 = find_overlap_indices(vertices_full_domain, cells_full_domain, x_min_S1, x_max_S1)

# Map global indices to local indices for each subdomain
local_indices_S2_in_S1 = map_to_local(overlap_S2_in_S1, cells_S1)
local_indices_S1_in_S2 = map_to_local(overlap_S1_in_S2, cells_S2)

A1_S2 = compute_global_A_1_A_2(vertices_S2, cells_S2)
A2_S2 = A1_S2.copy()

F2 = compute_Forcing(vertices_S2)

print(len(local_indices_S2_in_S1))

phi_left_u1_S2 = u[local_indices_S2_in_S1[:len(local_indices_S2_in_S1) // 2]]
phi_left_u2_S2 = u[local_indices_S2_in_S1[len(local_indices_S2_in_S1) // 2:]]

# Set boundary conditions for S2
set_boundary_conditions(n_cells_x_S2, n_cells_y, F2, [A1_S2, A2_S2], phi_left, phi_right, phi_bottom, phi_top)
# Left boundary

i_idx = np.zeros(n_cells_y + 1, dtype=np.int64)
j_idx = np.arange(0, n_cells_y + 1)

F2_first_half, F2_second_half = F2[:len(F2)//2], F2[len(F2)//2:]

A1_S2 = handle_Schwartz_boundary(i_idx, j_idx, n_cells_x_S2, F2_first_half, [A1_S2], phi_left_u1_S2)
A2_S2 = handle_Schwartz_boundary(i_idx, j_idx, n_cells_x_S2, F2_second_half, [A2_S2], phi_left_u2_S2)
F2 = np.concatenate((F2_first_half, F2_second_half))
S = scipy.sparse.bmat([[A1_S1, None], [None, A2_S1]]).tocsr()
u = scipy.sparse.linalg.spsolve(S, F1)
"""


"""X = vertices_S1[:, 0].reshape(n_cells_y+1, n_cells_x_S1+1)
Y = vertices_S1[:, 1].reshape(n_cells_y+1, n_cells_x_S1+1)
# Reshape u1 and u2 into 2D arrays
u1_2D = u1.reshape(n_cells_y+1, n_cells_x_S1+1)
u2_2D = u2.reshape(n_cells_y+1, n_cells_x_S1+1)

plot_results(X, Y, u1_2D, u2_2D, vertices_S1, u1, u2, n_cells_x_S1, n_cells_y)"""

286


c:\Users\leolu\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


'X = vertices_S1[:, 0].reshape(n_cells_y+1, n_cells_x_S1+1)\nY = vertices_S1[:, 1].reshape(n_cells_y+1, n_cells_x_S1+1)\n# Reshape u1 and u2 into 2D arrays\nu1_2D = u1.reshape(n_cells_y+1, n_cells_x_S1+1)\nu2_2D = u2.reshape(n_cells_y+1, n_cells_x_S1+1)\n\nplot_results(X, Y, u1_2D, u2_2D, vertices_S1, u1, u2, n_cells_x_S1, n_cells_y)'